In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
#from utils import INPUT_SHAPE, batch_generator
import argparse
import os

from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 240, 640, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)



Using TensorFlow backend.


In [2]:
def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    """
    Generate training image give image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center = image_paths[index]
            steering_angle = steering_angles[index]
            image = load_img(center)
            x = img_to_array(image)  
            x = x[240:480,:,:]
            image = array_to_img(x)
            # add the image and steering angle to the batch
            images[i] = image
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers



In [3]:
data_df = pd.read_csv('/home/sumedh/Desktop/Hardware/DrivingLog.csv')

X = data_df['center'].values
y = data_df['steering'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)



In [4]:
#batch_generator('', X_train, y_train, 16, True)
X_train.shape

(8568,)

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU, Input
from keras.layers import Conv2D, MaxPooling2D, Activation 
from keras.layers.convolutional import Convolution2D
from keras.models import Model
'''
model = Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(240,640,3)))
model.add(Conv2D(16, 17, 17, activation='elu', subsample=(2, 2)))
model.add(Conv2D(36, 13, 13, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 11, 11, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 5, 5, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))
model.summary()

'''
main_input = Input(shape=INPUT_SHAPE, dtype='float32', name='main_input')
lambda1 = Lambda(lambda x: x/127.5 - 1.,
            input_shape=INPUT_SHAPE,
            output_shape=INPUT_SHAPE)(main_input)
convShared1 = Convolution2D(16, 17, 17, subsample=(4, 4), border_mode="same")(lambda1)
eluShared1 = ELU()(convShared1)
convShared2 = Convolution2D(32, 11, 11, subsample=(2, 2), border_mode="same")(eluShared1)
eluShared2 = ELU()(convShared2)


SteeringConv = Convolution2D(64, 11, 11, subsample=(2, 2), border_mode="same")(eluShared2)
SteeringFlat = Flatten()(SteeringConv)
SteeringDrop1 = Dropout(.2)(SteeringFlat)
SteeringElu1 = ELU()(SteeringDrop1)
SteeringDense = Dense(512)(SteeringElu1)
SteeringDrop2 = Dropout(.5)(SteeringDense)
SteeringElu2 = ELU()(SteeringDrop2)
steering = Dense(1)(SteeringElu2)
model = Model(inputs=main_input, outputs= steering)
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 240, 640, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 240, 640, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 160, 16)       13888     
_________________________________________________________________
elu_1 (ELU)                  (None, 60, 160, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 80, 32)        61984     
_________________________________________________________________
elu_2 (ELU)          

/home/sumedh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (17, 17), strides=(4, 4), padding="same")`
/home/sumedh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (11, 11), strides=(2, 2), padding="same")`
/home/sumedh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (11, 11), strides=(2, 2), padding="same")`


In [6]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='auto')

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit_generator(batch_generator('', X_train, y_train, 32, True),
                    250,
                    10,
                    max_q_size=1,
                    validation_data=batch_generator('', X_valid, y_valid, 16, False),
                    nb_val_samples=len(X_valid),
                    #callbacks=[checkpoint],
                    verbose=1)


Instructions for updating:
Use tf.cast instead.


/home/sumedh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  app.launch_new_instance()
/home/sumedh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 250, 10, validation_data=<generator..., verbose=1, validation_steps=2856, max_queue_size=1)`
  app.launch_new_instance()


Epoch 1/10
250/250 [==============================] - 3332s 13s/step - loss: 2320.9209 - val_loss: 1652.7241
Epoch 2/10
250/250 [==============================] - 3306s 13s/step - loss: 1710.2604 - val_loss: 1483.0429
Epoch 3/10
250/250 [==============================] - 3244s 13s/step - loss: 1516.4567 - val_loss: 1285.2085
Epoch 4/10
250/250 [==============================] - 3247s 13s/step - loss: 1510.1854 - val_loss: 1275.1830
Epoch 5/10
250/250 [==============================] - 3245s 13s/step - loss: 1479.3125 - val_loss: 1172.1710
Epoch 6/10
250/250 [==============================] - 3244s 13s/step - loss: 1466.6539 - val_loss: 1287.4499
Epoch 7/10
250/250 [==============================] - 3244s 13s/step - loss: 1423.3260 - val_loss: 1171.5004
Epoch 8/10
250/250 [==============================] - 3246s 13s/step - loss: 1447.7216 - val_loss: 1217.2994
Epoch 9/10
250/250 [==============================] - 3246s 13s/step - loss: 1374.2172 - val_loss: 1218.7921
Epoch 10/10
250/250

In [10]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("steering.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("steering.h5")
print("Saved model to disk")



Saved model to disk


In [ ]:
model.fit_generator(batch_generator('', X_train, y_train, 32, True),
                    250,
                    10,
                    max_q_size=1,
                    validation_data=batch_generator('', X_valid, y_valid, 16, False),
                    nb_val_samples=len(X_valid),
                    #callbacks=[checkpoint],
                    verbose=1)


In [37]:
steers = []
a = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
counter = 0
for item in a:
    folderX = '/home/sumedh/Desktop/Hardware/Segmented/Run{}'.format(item)
    lenFolderX = len([f for f in os.listdir(folderX)])
    for i in range(lenFolderX):
        img = load_img('/home/sumedh/Desktop/Hardware/Segmented/Run{}/{}.jpg'.format(item, i+1))
        x = img_to_array(img)  
        x = x[240:480,:,:]
        image = array_to_img(x)
        image = np.asarray(image)       # from PIL image to numpy array

        image = np.array([image])       # the model expects 4D array
        #a = image[240:480,:,:]
        #print(image.shape)
        steering_angle = float(model.predict(image, batch_size=1))
        print(steering_angle)
        steers.append(steering_angle)

-24.834365844726562
-31.739246368408203
-29.249801635742188
-3.50034236907959
-9.60169506072998
-1.1629564762115479
-36.9595947265625
-32.12447738647461
-29.617835998535156
-36.166622161865234
-45.3392448425293
-53.04053497314453
-57.747066497802734
-82.45133972167969
-65.22511291503906
-45.221412658691406
-43.427284240722656
-45.66637420654297
-39.698795318603516
-44.23443603515625
-22.953161239624023
-38.95488357543945
-24.266189575195312
-18.709211349487305
-22.979154586791992
-14.648449897766113
-9.660325050354004
-11.097066879272461
-10.159090042114258
-10.53430461883545
-26.241207122802734
-32.24534225463867
-20.58881187438965
-19.54356575012207
-31.706289291381836
-32.9388427734375
-26.621166229248047
-23.782182693481445
-11.931344985961914
-18.8697509765625
-12.465948104858398
-15.049468994140625
-9.655640602111816
-16.356992721557617
-17.505430221557617
-22.74542808532715
-14.140483856201172
-16.37807846069336
-27.464447021484375
-39.13268280029297
-51.1163444519043
-47.257061

-3.8171629905700684
-3.386975049972534
-2.0624806880950928
-0.7104788422584534
-0.8784253597259521
1.3694016933441162
1.9191350936889648
10.256744384765625
8.635794639587402
12.044557571411133
19.874557495117188
17.31389045715332
21.21493148803711
32.1627082824707
40.248470306396484
55.375885009765625
69.98709106445312
91.37651062011719
94.80278015136719
78.26164245605469
78.9564208984375
85.72152709960938
88.78271484375
89.5407943725586
91.58480834960938
84.63921356201172
82.22256469726562
68.70880889892578
59.53396224975586
46.688777923583984
20.639606475830078
29.415239334106445
28.643192291259766
15.43301773071289
11.847915649414062
10.542377471923828
10.344088554382324
10.088831901550293
17.067394256591797
21.778730392456055
25.762914657592773
25.56454849243164
20.43268394470215
19.582866668701172
17.26786231994629
16.491243362426758
16.659469604492188
16.020891189575195
11.907049179077148
8.597001075744629
8.666818618774414
8.272768020629883
4.527768611907959
7.490859031677246
4.

-12.405584335327148
-10.132403373718262
-9.73765754699707
-8.430625915527344
0.7464887499809265
-4.176180839538574
-4.23274040222168
-2.0647237300872803
3.840698719024658
-7.5707478523254395
13.701171875
13.63288402557373
15.948349952697754
16.97766876220703
18.450660705566406
19.527925491333008
18.78026580810547
22.62282371520996
24.172565460205078
22.908384323120117
14.864439010620117
1.615481972694397
-10.158735275268555
-18.76064109802246
-18.886693954467773
-18.983715057373047
-37.968528747558594
-27.83633804321289
-25.658466339111328
-25.534250259399414
-23.182031631469727
-19.74140739440918
-14.330427169799805
-51.81401062011719
-37.73202133178711
-40.658050537109375
-47.82441329956055
-46.45055389404297
-46.02308654785156
-48.074310302734375
-59.65502166748047
-62.687232971191406
-62.14360427856445
-57.42635726928711
-52.35250473022461
-44.663421630859375
-34.18259811401367
-48.44883728027344
-52.25562286376953
-50.94863510131836
-69.06504821777344
-78.93025970458984
-94.731498

46.74409866333008
51.0098991394043
50.0989875793457
48.8569450378418
45.45503616333008
36.89846420288086
36.448463439941406
36.09193420410156
37.55903625488281
34.730709075927734
33.5790901184082
27.4315128326416
22.188039779663086
13.01870059967041
11.5170316696167
13.700051307678223
8.33454418182373
5.560617923736572
2.8606600761413574
-0.3743705749511719
-0.34743666648864746
1.5305283069610596
6.753626823425293
13.555747985839844
18.389511108398438
20.541950225830078
23.454328536987305
23.611425399780273
24.60728645324707
22.5388240814209
21.363372802734375
24.46902847290039
35.47542953491211
51.67372512817383
70.32148742675781
82.41371154785156
80.80460357666016
80.49225616455078
101.5103759765625
103.86272430419922
113.65399169921875
107.1042251586914
114.6341552734375
104.96057891845703
93.12334442138672
83.25922393798828
89.03343200683594
73.01338195800781
62.05118942260742
66.3670883178711
55.02821731567383
28.67931365966797
18.777477264404297
-3.9117350578308105
-1.88609051704

-22.038326263427734
-17.129976272583008
-15.380337715148926
-11.433279991149902
-8.55608081817627
-2.253504991531372
4.067985534667969
10.009529113769531
13.105814933776855
11.845016479492188
10.117485046386719
10.741556167602539
12.578521728515625
14.391661643981934
13.163963317871094
8.933976173400879
8.210095405578613
7.7527079582214355
9.15943431854248
9.065340995788574
3.1284842491149902
13.930816650390625
8.334386825561523
-1.6478161811828613
0.020845891907811165
1.7276712656021118
14.368775367736816
20.343441009521484
8.623982429504395
9.052852630615234
14.298295021057129
-1.0884346961975098
7.485482692718506
6.047420978546143
4.309337139129639
-15.28763484954834
14.727181434631348
23.765012741088867
8.519832611083984
16.65130043029785
3.6782755851745605
-21.042680740356445
-44.19512939453125
-51.22964096069336
-54.23308563232422
-43.95797348022461
-32.430843353271484
-20.06593894958496
-13.995967864990234
-8.801422119140625
-12.049962997436523
-6.858879566192627
-7.113688468933

...

17.76220703125
7.316290855407715
8.351797103881836
8.065959930419922
4.176153659820557
8.349164009094238
10.401483535766602
7.386674880981445
3.6657187938690186
2.3475966453552246
-3.6124682426452637
-4.927773475646973
-20.539039611816406
-32.71983337402344
-32.73371124267578
-31.73726463317871
-36.28034973144531
-37.869873046875
-39.06658172607422
-44.165565490722656
-47.216285705566406
-49.5820198059082
-53.692100524902344
-50.346282958984375
-62.176414489746094
-61.27801513671875
-62.367950439453125
-67.41368103027344
-67.06370544433594
-68.06710052490234
-64.95549011230469
-65.18872833251953
-60.23898696899414
-56.955081939697266
-52.4349250793457
-52.82046127319336
-53.5003662109375
-57.337364196777344
-56.24153518676758
-63.26222610473633
-68.00265502929688
-67.54337310791016
-67.76570129394531
-72.62520599365234
-61.77299880981445
-69.52587890625
-41.172142028808594
-46.013999938964844
-35.387332916259766
-34.73616027832031
-45.16532516479492
-28.53931999206543
-31.4210662841796

68.17587280273438
84.40187072753906
88.49993133544922
98.30975341796875
102.20276641845703
98.7985610961914
74.23893737792969
72.7037124633789
74.68404388427734
71.9224624633789
41.29249572753906
30.239151000976562
6.2483601570129395
1.5660767555236816
5.6428141593933105
1.7658116817474365
-16.69672393798828
-20.882320404052734
-23.201658248901367
-24.81816291809082
-22.954071044921875
-20.4183406829834
-8.269295692443848
-11.637648582458496
-4.11384391784668
-12.786202430725098
-13.447367668151855
-17.32473373413086
-25.167301177978516
-43.86268615722656
-17.308975219726562
-39.821102142333984
-34.058555603027344
-33.0494270324707
-33.818668365478516
-30.159042358398438
-38.549861907958984
-38.20687484741211
-37.595985412597656
-40.19154739379883
-45.221275329589844
-34.92658615112305
-34.59075927734375
-37.91304397583008
-29.485931396484375
-42.84435272216797
-29.33980369567871
-32.00123596191406
-43.517269134521484
-26.828344345092773
-27.342594146728516
-31.468000411987305
-13.9976

-35.35724639892578
-32.89882278442383
-30.0411319732666
-28.56755256652832
-19.735071182250977
-8.31623363494873
-0.8394327759742737
1.521464228630066
5.455099105834961
10.407356262207031
10.248835563659668
18.04584503173828
13.940500259399414
12.886129379272461
11.861071586608887
7.791322231292725
5.042272567749023
7.071394920349121
0.9964228868484497
-1.762082815170288
-1.3399049043655396
-0.9413940906524658
-0.02428126148879528
-3.5415093898773193
-1.750510811805725
-1.488433599472046
-0.8243744373321533
3.3438239097595215
-1.7806986570358276
-0.5983006954193115
-1.5154087543487549
0.6186549663543701
4.242882251739502
4.616918087005615
9.900221824645996
15.372760772705078
17.572519302368164
30.444988250732422
29.62373924255371
32.10517883300781
30.789691925048828
28.703033447265625
27.003658294677734
28.02323341369629
21.029399871826172
17.753562927246094
14.484610557556152
11.485843658447266
7.907981872558594
6.957928657531738
9.831249237060547
10.960719108581543
12.284588813781738

-27.68048858642578
-31.37738800048828
-22.550792694091797
-29.059484481811523
-34.36399459838867
-30.99102020263672
-23.07853126525879
-24.681270599365234
-22.616138458251953
-21.338199615478516
-24.649606704711914
-27.70802879333496
-29.17896842956543
-29.57975959777832
-29.897626876831055
-35.85820770263672
-42.62994384765625
-45.1243782043457
-47.67369842529297
-51.26786804199219
-54.395057678222656
-56.14179229736328
-58.20008850097656
-57.22089767456055
-55.16020202636719
-53.87843704223633
-57.973175048828125
-58.94939041137695
-60.445953369140625
-65.71611022949219
-67.48568725585938
-67.12203979492188
-61.81599426269531
-53.36946105957031
-53.2470703125
-45.74748992919922
-45.29514694213867
-40.26107406616211
-34.07611846923828
-42.47684860229492
-41.656768798828125
-45.719764709472656
-40.9443244934082
-26.176372528076172
-26.982141494750977
-16.56239891052246
4.670567989349365
-13.922368049621582
-13.582446098327637
-1.2945756912231445
-1.3541592359542847
6.120458126068115
7.

72.14513397216797
70.55829620361328
58.128135681152344
52.475013732910156
64.05500030517578
36.909175872802734
25.408674240112305
25.937496185302734
18.72441291809082
10.091391563415527
13.035740852355957
-4.853666305541992
-5.2936553955078125
-5.435884475708008
-13.573945999145508
-15.79023265838623
-19.4316463470459
-15.781188011169434
-16.122726440429688
2.40041446685791
-10.833681106567383
-19.624711990356445
-14.864967346191406
-14.82502555847168
-12.835721015930176
-8.654260635375977
-4.561628818511963
-6.087160587310791
-8.012941360473633
-9.656492233276367
-13.483809471130371
-29.384700775146484
-18.26796531677246
-21.62264060974121
-20.80457878112793
-36.56656265258789
-33.674381256103516
-33.4400520324707
-23.57975196838379
-7.517039775848389
-15.162616729736328
-11.169360160827637
-7.800539016723633
-3.9235055446624756
3.489189863204956
6.647406101226807
10.194552421569824
16.815507888793945
21.20720100402832
26.731782913208008
24.40454864501953
31.890769958496094
34.8942527

In [38]:
import csv

actual = []
a = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
counter = 0
for item in a:
    folderX = '/home/sumedh/Desktop/Hardware/NewData/Run{}'.format(item)
    lenFolderX = len([f for f in os.listdir(folderX)])
    with open(folderX + "/" + 'Run{}.csv'.format(item),'rt')as f:
        data = csv.reader(f)
        for row in data:
            print(row[1])
            actual.append(float(row[1]))

-22
-3
49
90
102
96
77
62
0
-34
-62
-83
-99
-105
-83
-40
-19
-12
-3
-3
0
0
-3
-37
-37
-37
-37
-37
-37
-37
-68
-83
-71
-34
-19
-9
25
68
90
90
65
52
49
46
40
25
3
-12
-25
-34
-43
-46
-49
-46
-34
-31
-28
-25
-25
-25
-22
-19
-15
-12
-9
-9
-9
-9
-9
-9
43
90
108
93
86
65
31
15
12
12
6
3
0
-15
-19
-19
-19
-19
-19
-19
-56
-65
-62
-59
-65
-62
-43
-40
-40
-40
-40
-40
-40
-40
-40
-40
-40
-40
-40
-40
-37
-6
46
74
83
74
74
74
74
74
59
52
43
34
25
12
9
6
6
6
9
31
46
49
52
52
52
52
52
52
40
37
40
40
40
40
40
40
40
40
59
80
93
90
83
71
62
56
56
56
46
43
43
43
43
43
43
43
43
43
46
49
68
80
80
68
59
56
65
65
86
93
102
99
90
83
65
9
9
9
3
6
9
12
19
37
37
37
37
37
90
108
114
102
74
59
52
52
52
52
56
96
99
71
56
46
37
37
37
37
28
25
28
31
34
34
34
34
34
34
37
40
49
65
86
93
77
77
77
77
59
49
43
46
83
102
105
102
102
102
90
86
37
25
22
62
99
117
117
117
124
130
71
25
15
9
6
6
6
6
3
-3
-9
-15
-19
-22
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-19
-12
-9
-6
-6


-3
0
3
3
3
3
3
6
9
12
15
19
28
34
34
34
34
37
40
43
46
56
59
65
71
71
71
-6
-62
-46
-34
-25
-15
-6
34
-6
34
68
77
74
31
-22
-46
-68
-83
-90
-90
-93
-90
-83
-74
-68
-62
-59
-59
-59
-59
-56
-46
-43
-31
-12
-3
6
15
15
15
25
19
-22
-25
-22
-34
-37
-37
-37
-37
-34
-31
-22
-19
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-15
-40
-49
-59
-68
-71
-77
-80
-80
-80
-71
-65
-59
-56
-52
-49
-49
-49
-49
-49
-46
-43
-6
22
43
52
56
56
56
56
59
49
3
-34
-40
-46
-56
-65
-65
-65
-62
-56
-52
-49
-43
-25
3
40
77
77
96
99
96
93
59
56
52
15
15
15
-3
-6
-3
43
86
105
102
102
102
102
105
102
56
31
25
25
25
25
25
25
34
83
102
99
99
99
99
99
99
99
56
15
0
-6
-12
-19
-22
-43
-52
-62
-62
-56
-49
-19
31
62
68
68
68
68
65
59
56
62
65
68
74
74
74
74
77
93
99
99
77
68
59
31
31
31
9
3
0
0
0
0
0
0
0
0
0
12
15
12
12
12
12
12
12
12
22
40
59
40
12
6
28
25
9
0
-15
-19
-22
-31
-49
-65
-65
-65
-65
-65
-74
-80
-83
-80
-80
-80
-80
-80
-80
-80
-77
-71
-68
-62
-52
-43
-40
-25
31
62
83
49
19
0
-12
-28
-52
-62
-62

52
74
86
86
86
86
86
96
86
74
62
59
56
46
40
31
31
28
25
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
31
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
12
-15
-31
-40
-52
-71
-71
-71
-71
-74
-80
-83
-86
-86
-86
-86
-86
-86
-86
-83
-80
-77
-77
-77
-77
-77
-77
-77
-77
-74
-71
-62
-56
-56
-56
-56
-56
-56
-56
-52
-12
3
0
0
0
0
0
0
0
-6
-9
-19
-25
-25
-25
-25
-25
-25
-25
-28
-31
-37
-40
-40
-56
-56
-56
-56
-56
-83
-99
-105
-114
-117
-114
-114
-114
-114
-114
-114
-111
-108
-108
-108
-108
-108
-108
-108
-108
-105
-102
-62
-62
-62
-62
-62
-62
-62
-62
-40
-37
-40
-40
-40
-40
-40
-40
-40
-40
-43
-46
-93
-99
-102
-99
-96
-96
-96
-96
-90
-86
-83
-65
-49
-43
-40
-34
-15
-15
-3
0
6
25
49
77
108
127
148
154
151
108
49
15
-19
-37
-46
-43
-43
-43
-25
-19
-15
-12
-6
-3
3
19
19
19
46
65
83
96
105
99
71
77
31
31
0
-25
-28
-25
9
80
105
1

-77
-77
-74
-68
-59
-37
-37
-37
-37
-37
-37
-37
-31
-28
-25
-28
-28
-28
-28
-28
-28
-28
-40
-56
-77
-93
-99
-99
-99
-99
-99
-99
-99
-96
-93
-90
-74
-62
-65
-65
-65
-65
-68
-77
-80
-77
-77
-77
-77
-77
-77
-77
-65
-59
-56
-52
-52
-52
-52
-52
-52
-52
-49
-37
-37
-37
-37
-37
-37
-37
-37
-37
-31
-28
-19
-3
9
15
22
31
37
37
40
56
62
65
68
74
80
80
80
80
65
62
62
62
62
62
62
62
62
62
59
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
43
19
6
0
0
0
0
0
0
0
0
0
3
6
22
22
22
22
22
22
28
31
34
52
68
68
68
68
68
68
80
80
80
80
77
74
71
71
71
71
71
43
31
28
28
28
28
28
28
28
25
28
31
37
37
37
37
37
37
37
31
6
-22
-22
-22
-22
-22
-22
-22
-22
-40
-46
-43
-31
-28
-25
-19
-15
-15
-15
-12
-6
-3
0
0
3
6
6
6
6
25
40
49
56
59
62
59
59
59
59
56
52
43
37
34
34
34
34
34
34
31
28
28
28
28
28
28
28
28
28
19
15
19
19
19
19
19
19
19
19
22
22
22
22
22
22
22
22
22
22


In [35]:
print(type(actual), type(steers))
print(type(actual[0]), type(steers[0]))


<class 'list'> <class 'list'>
<class 'float'> <class 'float'>


In [39]:
from sklearn.metrics import r2_score
#actual = np.asarray(actual)       # from PIL image to numpy array
#steers = np.asarray(steers)       # from PIL image to numpy array
print(type(actual), type(steers))
print(r2_score(actual, steers))


<class 'list'> <class 'list'>
0.6499482752443319
